In [62]:
import pandas as pd
from datetime import datetime, timedelta

import sqlite3
import random
import os
from faker import Faker

In [64]:
df_rates = pd.read_csv("eur_historical_rates_2020_2025.csv")
df_rates = df_rates.melt(id_vars=['date'], var_name='currency', value_name='rate')
df_rates.rename(columns={'date': 'Month'}, inplace=True)
df_rates = df_rates[['Month', 'currency', 'rate']]

#get rates for specific currencies and months
def get_rate(currency, month, df_rates=df_rates):
    """    Get the exchange rate for a specific currency and month.
    Args:
        currency (str): The currency code (e.g., 'USD', 'KES').
        month (datetime): The month for which to get the rate.
        df_rates (DataFrame): DataFrame containing exchange rates.
    Returns:
        float: The exchange rate for the specified currency and month, or None if not found.
    """
    # Find the last day of the given month
    next_month = month.replace(day=28) + timedelta(days=4)  # this will never fail
    last_day = next_month - timedelta(days=next_month.day)
    month_str = last_day.strftime("%Y/%m/%d")
    row = df_rates[(df_rates['currency'] == currency) & (df_rates['Month'].str.startswith(month_str))]
    if not row.empty:
        return row['rate'].values[0]
    else:
        return None  # or raise an error if preferred
    
# Assert exchange rates for specific currencies and months
assert abs(get_rate("KES", datetime(2024, 1, 1),df_rates) - 175.28479265) < 1e-6, "KES rate for 2024-01 does not match"
assert abs(get_rate("XOF", datetime(2023, 1, 1),df_rates) - 655.957) < 1e-6, "XOF rate for 2023-01 does not match"
assert abs(get_rate("USD", datetime(2023, 1, 1),df_rates) - 1.08513702) < 1e-8, "USD rate for 2023-01 does not match"


In [66]:
# Initialisation
faker = Faker()

# For reproducibility
random.seed(68) 
faker.seed_instance(68)

projects = [
    {"id": "KE01", "name": "Nairobi Coordo", "country": "Kenya", "currency": "KES"},
    {"id": "KEO2", "name": "Nairobi Emmergency", "country": "Kenya", "currency": "KES"},
    {"id": "BE01", "name": "Brussels Coordo", "country": "Belgium", "currency": "EUR"},
    {"id": "BE55", "name": "Namur Covid19", "country": "Belgium", "currency": "EUR"},
    {"id": "BF01", "name": "Ouagadougou Coordo", "country": "Burkina Faso", "currency": "XOF"},
    {"id": "BF02", "name": "Koudougou HIV", "country": "Burkina Faso", "currency": "XOF"},
    {"id": "SN01", "name": "Dakkar Coordo", "country": "Senegal", "currency": "XOF"},
    {"id": "SN02", "name": "Louga Primary Health Care", "country": "Senegal", "currency": "XOF"}
]
categories = {
    "HR": [{"name": "Salaries"}, {"name": "Training"}, {"name": "Recruitment"}],
    "Medical": [
        {"name": "Medications"},
        {"name": "Medical Equipment", "investment": True},
        {"name": "Consultations"}
    ],
    "Logistics": [
        {"name": "Fuel"},
        {"name": "Transport"},
        {"name": "Vehicle Maintenance"}
    ],
    "Supply": [
        {"name": "Supplies"},
        {"name": "Warehousing"},
        {"name": "Cold Chain"}
    ],
    "Operations": [
        {"name": "Field Operations"},
        {"name": "Community Engagement"},
        {"name": "Monitoring"}
    ]
}
years = list(range(2020, 2026))
months = [f"{m:02d}" for m in range(1, 13)]


# Create a DataFrame from the projects list and export to Excel
df_projects = pd.DataFrame(projects)
os.makedirs("data", exist_ok=True)
df_projects.to_excel(os.path.join("data", "projects.xlsx"), index=False)


# Function to generate and export project, expenses, and budget data to 3 xlsx files (no SQLite, English comments)
def create_project_xlsx(project,df_rates=df_rates):
    """    Create and export project, expenses, and budget data to Excel files.
    Args:
        project (dict): Project information containing id, name, country, and currency.
        df_rates (DataFrame): DataFrame containing exchange rates.
    """

    os.makedirs("data", exist_ok=True)

    # Prepare expenses and budget data
    expenses_rows = []
    budget_rows = []
    for year in years:
        for month in months:
            
            for dept, cats in list(categories.items())[:4]:
                for cat in cats:
                    # Budget version 1
                    # Make budgets higher for the last 3 months of the year
                    if int(month) in [10, 11, 12]:
                        budget = round(random.uniform(4000, 8000), 2)
                    else:
                        budget = round(random.uniform(1000, 5000), 2)
                    budget_rows.append({
                        "year": year,
                        "month": month,
                        "department": dept,
                        "category": cat['name'],
                        "budget_eur": budget,
                        "version": "v1"
                    })

                    # Actual expense (80% to 110% of budget)
                    multiplier = random.uniform(0.8, 1.1)
                    amount_eur = round(budget * multiplier, 2)
                    rate = None
                    # Use df_rates DataFrame for exchange rate lookup
                    date_str = f"{year}/{month}/31"
                    rate_row = df_rates[(df_rates['Month'] == date_str) & (df_rates['currency'] == project['currency'])]
                    if not rate_row.empty and pd.notnull(rate_row.iloc[0]['rate']):
                        rate = rate_row.iloc[0]['rate']
                    if rate is not None and rate != 0:
                        amount_local = amount_eur * rate
                    else:
                        amount_local = None

                    expenses_rows.append({
                        "year": year,
                        "month": month,
                        "department": dept,
                        "category": cat['name'],
                        "amount_local": amount_local,
                        "currency": project['currency'],
                        "amount_eur": amount_eur
                    })

    # Export to Excel files
    expenses_df = pd.DataFrame(expenses_rows)
    expenses_df.to_excel(os.path.join("data", f"{project['id']}_expenses.xlsx"), index=False)
    budget_df = pd.DataFrame(budget_rows)
    budget_df.to_excel(os.path.join("data", f"{project['id']}_budget.xlsx"), index=False)
    
# Execution for each project
for project in projects:
    create_project_xlsx(project,df_rates)




In [68]:
#merge budget files and create column for country
import os
import pandas as pd

# Paths
data_dir = "data"
projects_file = os.path.join(data_dir, "projects.xlsx")

# Read project metadata
df_projects = pd.read_excel(projects_file)

# Dictionary to map project ID to country
project_country_map = df_projects.set_index("id")["country"].to_dict()

# Collect all budget files
budget_files = [f for f in os.listdir(data_dir) if f.endswith("_budget.xlsx")]

# List to hold each budget DataFrame
budget_dfs = []

# Process each budget file
for file in budget_files:
    project_id = file.split("_")[0]  # Extract ID from filename
    df = pd.read_excel(os.path.join(data_dir, file))
    df["project_id"] = project_id
    df["country"] = project_country_map.get(project_id, "Unknown")
    budget_dfs.append(df)

# Combine all into one DataFrame
df_all_budget = pd.concat(budget_dfs, ignore_index=True)

# Export to Excel
merged_budget_path = os.path.join(data_dir, "merged_budget.xlsx")
df_all_budget.to_excel(merged_budget_path, index=False)

print(f"Merged budget saved to {merged_budget_path}")

Merged budget saved to data\merged_budget.xlsx


In [70]:
#merge expenses files
import os
import pandas as pd

# Paths
data_dir = "data"
projects_file = os.path.join(data_dir, "projects.xlsx")

# Load project metadata
df_projects = pd.read_excel(projects_file)
project_country_map = df_projects.set_index("id")["country"].to_dict()

# Find all expenses files
expenses_files = [f for f in os.listdir(data_dir) if f.endswith("_expenses.xlsx")]

# Store all expense records
expenses_dfs = []

for file in expenses_files:
    project_id = file.split("_")[0]
    df = pd.read_excel(os.path.join(data_dir, file))
    df["project_id"] = project_id
    df["country"] = project_country_map.get(project_id, "Unknown")
    
    # Optional: warn if project_id is missing
    if df["country"].iloc[0] == "Unknown":
        print(f"Warning: Country not found for project ID '{project_id}' in file '{file}'")

    expenses_dfs.append(df)

# Combine all into one DataFrame
df_all_expenses = pd.concat(expenses_dfs, ignore_index=True)

# Save to Excel
output_file = os.path.join(data_dir, "merged_expenses.xlsx")
df_all_expenses.to_excel(output_file, index=False)

print(f"Merged expenses saved to {output_file}")


Merged expenses saved to data\merged_expenses.xlsx
